In [870]:
import os
import zipfile
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import cv2


import torch
import torch.nn as nn
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

### Dataset и Dataloader

In [871]:
from dataloader import LipReadingVideoDataset, LipReadingVideoDatasetZIP, collate_func

In [818]:
#data_path = os.path.normpath('F:/lip_reading_data/LRS3/lrs3_trainval')
data_path_zip = os.path.normpath('../data/lrs3_trainval.zip')

In [882]:
from torchvideotransforms import video_transforms

flip = video_transforms.RandomHorizontalFlip(p=0.5)
rotation = video_transforms.RandomRotation(degrees=15)

transform = video_transforms.Compose([flip, rotation])

In [883]:
#ds = LipReadingVideoDataset(data_path)
ds_zip = LipReadingVideoDatasetZIP(data_path_zip, transform=transform)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


In [884]:
dl = torch.utils.data.DataLoader(ds_zip, batch_size=4, shuffle=True, collate_fn=collate_func)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


In [885]:
for i, (vframes_batch, list_of_sentence) in enumerate(dl):
    print(vframes_batch.shape)
    if i >=1:
        break

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives w

torch.Size([4, 3, 128, 3, 224, 224])


/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives w

torch.Size([4, 3, 128, 3, 224, 224])


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: dl, ds_zip
  warnings.warn(message)


### Показ видео

In [866]:
sample = ds_zip[1]
vframes, text, txt_path, video_path = sample.values()

/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/usr/local/lib/python3.7/dist-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: ds_zip
  warnings.warn(message)


In [771]:
# не работает
def video_show(vframes):
    time, C, H, W = vframes.shape
    for i in range(time):
        frame = vframes[i].numpy()
        
        cv2.imshow('frame', frame)
        # & 0xFF is required for a 64-bit system
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

### Подготовка словаря

In [ ]:
sentences = []

for dir_path, dir_names, file_names in os.walk(data_path):
    # перебрать файлы
    for file_name in file_names:
        if file_name.endswith('.txt'):
            txt_path = os.path.join(dir_path, file_name)
            txt_file = open(txt_path, 'r')
            
            _, text = txt_file.readline().split(':') # прочитанный текст
            text = text.strip()
            
            sentences.append(text)
            txt_file.close()

In [886]:
file = open('sentences.txt', 'r')
sentences = list(map(lambda line: line.strip(), file.readlines()))
file.close()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:809: UserWarning: The following variables cannot be serialized: file
  warnings.warn(message)


In [887]:
sentences[:5]

['ONE DAY A YOUNG BOY COMES UPON THE SUNFLOWER WHILE VISITING THE GARDEN AND HE NOTICES HOW WEAK IT LOOKS',
 'JUST LIKE REACHING OUT TO THE SUNFLOWER BY PROVIDING SOMEONE WHO IS NEGLECTED ISOLATED OR FORGOTTEN',
 'THEY ARE INCREDIBLE YET SO OFTEN THEY ARE FORGOTTEN LACKING THE LOVE AND APPRECIATION THEY DESERVE',
 'I WILL HAVE HOPEFULLY MET MY GOAL',
 'THAT WAS COMPLETELY WRONG']

In [888]:
from collections import Counter
import nltk

tokenizer = nltk.tokenize.WordPunctTokenizer()
#tokenizer = nltk.tokenize.TreebankWordTokenizer()

token_counts = Counter(tokenizer.tokenize(' '.join(sentences)))

In [889]:
min_count = 1

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = []
for key, values in token_counts.items():
    if values >= min_count:
        tokens.append(key)

# Add a special tokens for unknown and empty words
BOS, EOS, UNK, PAD = '_BOS_', '_EOS_', '_UNK_', '_PAD_'
tokens = [PAD, BOS, EOS, UNK] + tokens

In [890]:
len(tokens)

17193

In [891]:
# словари
token_to_id = {tok: i for i, tok in enumerate(tokens)}
id_to_token = {i: tok for tok, i in token_to_id.items()}

### Network

In [892]:
vframes_batch.shape

torch.Size([4, 3, 128, 3, 224, 224])

In [893]:
list_of_sentences

['IT IS TIME FOR US TO REMOVE THE BARRIERS THAT PERMANENTLY RELEGATE INDIVIDUALS WITH CRIMINAL',
 'BUT WHERE DO THOSE LIMITS COME FROM',
 "IT'S A RIGHT ENSHRINED BY THE UNITED NATIONS",
 'BUT JUST A REMINDER']

In [894]:
def sentences_to_tokens(list_of_sentences, tokenaizer):
    list_of_tokens = []
    for sentence in list_of_sentences:
        list_of_tokens.append(tokenizer.tokenize(sentence))
    return list_of_tokens

In [895]:
list_of_tokens = sentences_to_tokens(list_of_sentences, tokenizer)

In [689]:
list_of_tokens

[['IT',
  'IS',
  'TIME',
  'FOR',
  'US',
  'TO',
  'REMOVE',
  'THE',
  'BARRIERS',
  'THAT',
  'PERMANENTLY',
  'RELEGATE',
  'INDIVIDUALS',
  'WITH',
  'CRIMINAL'],
 ['BUT', 'WHERE', 'DO', 'THOSE', 'LIMITS', 'COME', 'FROM'],
 ['IT', "'", 'S', 'A', 'RIGHT', 'ENSHRINED', 'BY', 'THE', 'UNITED', 'NATIONS'],
 ['BUT', 'JUST', 'A', 'REMINDER']]

### Network

In [911]:
from network import LipReadinNN

In [912]:
my_network = LipReadinNN(token_to_id)

In [913]:
my_network.train(True)

with torch.no_grad():
    output = my_network(vframes_batch, list_of_tokens)

In [914]:
out_sent = list(map(lambda l: ' '.join(l), output))

In [915]:
out_sent

['325 PRIMES PRIMES ACQUIRED ACQUIRED PRIMES PRIMES CURSE SEQUENTIAL PRIMES WEAPON COMPLEXITY FESTIVAL SPACESHIP PRIMES ACQUIRED CATERING',
 'PRIMES PRIMES SPACESHIP PRIMES PRIMES PRIMES PRIMES PRIMES PRIMES SPACESHIP SPACESHIP PRIMES ACQUIRED PRIMES PRIMES SPACESHIP PRIMES',
 'PRIMES WEDDING PRIMES INSUFFICIENT PRIMES SCALED PRIMES PRIMES RESPECTFULLY ACQUIRED RAPES SCALED SCALED ACQUIRED 325 PRIMES VOLUNTARY',
 'PRIMES PRESTIGIOUS DONORSHIP PRIMES PRIMES PRIMES PRIMES PRIMES UPSTAIRS PRIMES PRIMES BATTLEFIELD CARPENTERS PRIMES PRIMES PRIMES CHASING']

In [916]:
my_network.loss

tensor(9.9465)